In [1]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

StatementMeta(, 90455b6c-e3c3-48c8-9fc0-1715273bbfba, 3, Finished, Available, Finished)

DataFrame[]

In [9]:
from pyspark.sql.functions import col, to_date, count, sum as _sum, avg, round

df_silver = spark.table("silver.nyc_taxi_trips_silver")

df_gold = (
    df_silver
    .withColumn("trip_date", to_date(col("tpep_pickup_datetime")))
    .groupBy("trip_date", "Borough")
    .agg(
        count("*").alias("total_trips"),
        round(_sum("total_amount"), 2).alias("total_revenue"),
        round(avg("trip_distance"), 2).alias("avg_trip_distance"),
        round(avg("fare_amount"), 2).alias("avg_fare_amount")
    )
    .orderBy("trip_date", "Borough")
)

df_gold.show(10)

StatementMeta(, 2d1e0f75-6cea-4565-96b6-297354a0722e, 11, Finished, Available, Finished)

+----------+---------+-----------+-------------+-----------------+---------------+
| trip_date|  Borough|total_trips|total_revenue|avg_trip_distance|avg_fare_amount|
+----------+---------+-----------+-------------+-----------------+---------------+
|2007-12-05|Manhattan|          5|       113.75|              3.0|           17.0|
|2009-01-01|   Queens|          5|       351.95|            10.77|           52.7|
|2024-12-31|Manhattan|         80|       1993.1|             2.66|          16.48|
|2024-12-31|   Queens|         25|       952.75|             6.84|           27.6|
|2025-01-01|    Bronx|       1735|      35570.5|           209.48|          17.15|
|2025-01-01| Brooklyn|      11935|     303846.0|             5.42|          20.35|
|2025-01-01|      EWR|         25|       1542.0|              1.7|          56.68|
|2025-01-01|Manhattan|     344610|    8150203.8|             2.79|          16.22|
|2025-01-01|      N/A|        230|      18581.6|            11.89|          73.38|
|202

In [10]:
(
    df_gold.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("gold.nyc_taxi_kpi_by_date_borough")
)


StatementMeta(, 2d1e0f75-6cea-4565-96b6-297354a0722e, 12, Finished, Available, Finished)

In [12]:
spark.sql("""
SELECT *
FROM gold.nyc_taxi_kpi_by_date_borough
ORDER BY trip_date, Borough
LIMIT 15
""").show()


StatementMeta(, 2d1e0f75-6cea-4565-96b6-297354a0722e, 14, Finished, Available, Finished)

+----------+-------------+-----------+-------------+-----------------+---------------+
| trip_date|      Borough|total_trips|total_revenue|avg_trip_distance|avg_fare_amount|
+----------+-------------+-----------+-------------+-----------------+---------------+
|2007-12-05|    Manhattan|          5|       113.75|              3.0|           17.0|
|2009-01-01|       Queens|          5|       351.95|            10.77|           52.7|
|2024-12-31|    Manhattan|         80|       1993.1|             2.66|          16.48|
|2024-12-31|       Queens|         25|       952.75|             6.84|           27.6|
|2025-01-01|        Bronx|       1735|      35570.5|           209.48|          17.15|
|2025-01-01|     Brooklyn|      11935|     303846.0|             5.42|          20.35|
|2025-01-01|          EWR|         25|       1542.0|              1.7|          56.68|
|2025-01-01|    Manhattan|     344610|    8150203.8|             2.79|          16.22|
|2025-01-01|          N/A|        230|     